# Prediction of Wild Blueberry Yield
[Blueberry Yield- Playground Series S3E14](https://www.kaggle.com/competitions/playground-series-s3e14)


    - 'clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange',
       'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange',
       'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds',
       'yield', 'dataset'

- Yield
- Dataset: 1 is current (synthethic), 2 is [original](https://www.kaggle.com/datasets/shashwatwork/wild-blueberry-yield-prediction-dataset)

In [14]:

import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna

data_train = pd.read_csv("/home/rainer/Downloads/ML_datasets/blueberry_train.csv", index_col="id")
data_train["dataset"]=1
data_old = pd.read_csv("/home/rainer/Downloads/ML_datasets/WildBlueberryPollinationSimulationData.csv")
data_old = data_old.drop('Row#', axis=1)
data_old["dataset"]=2
data_train = pd.concat([data_train, data_old])

In [15]:
data_train.dtypes

clonesize               float64
honeybee                float64
bumbles                 float64
andrena                 float64
osmia                   float64
MaxOfUpperTRange        float64
MinOfUpperTRange        float64
AverageOfUpperTRange    float64
MaxOfLowerTRange        float64
MinOfLowerTRange        float64
AverageOfLowerTRange    float64
RainingDays             float64
AverageRainingDays      float64
fruitset                float64
fruitmass               float64
seeds                   float64
yield                   float64
dataset                   int64
dtype: object

In [16]:
X, Y = data_train.drop('yield', axis=1), data_train[['yield']]
dtrain = xgb.DMatrix(X, Y, enable_categorical=False)

In [17]:
boostrounds=200

def objective(trial):
    param = {
        'tree_method':'gpu_hist',
        "objective": "reg:squarederror", #https://xgboost.readthedocs.io/en/stable/parameter.html#learning-task-parameters
        'eta': trial.suggest_float('eta', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 10, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),
    }


    results = xgb.cv(
        params=param,
        dtrain=dtrain,
        num_boost_round=boostrounds,
        nfold=5,
        early_stopping_rounds=50,
        verbose_eval=100, # print eval every xth round
        metrics=["rmse", "mae"], #see "eval_metric"
    )

    return results.iloc[-1]["test-rmse-mean"]

In [18]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-05-03 17:43:29,542] A new study created in memory with name: no-name-b8fdc0f7-811f-41ee-af8e-9c700e1e529b


[0]	train-rmse:4811.13275+4.53622	train-mae:4682.82258+5.36577	test-rmse:4811.78692+18.43946	test-mae:4683.22415+22.15297
[70]	train-rmse:481.48381+5.38521	train-mae:310.11453+2.94406	test-rmse:560.41530+18.36313	test-mae:351.84377+7.30208


[I 2023-05-03 17:43:31,408] Trial 0 finished with value: 557.6661985761706 and parameters: {'eta': 0.2229797640735916, 'max_depth': 5, 'min_child_weight': 0.016228665390832713, 'subsample': 0.6007194738642312, 'colsample_bynode': 0.6641392501111079}. Best is trial 0 with value: 557.6661985761706.


[0]	train-rmse:4284.14078+3.43669	train-mae:4156.91954+4.65432	test-rmse:4284.49557+17.89544	test-mae:4156.87240+21.38950
[68]	train-rmse:433.13018+5.46685	train-mae:283.05300+2.92560	test-rmse:571.38640+18.32055	test-mae:360.70316+6.81964


[I 2023-05-03 17:43:33,133] Trial 1 finished with value: 560.3156208852898 and parameters: {'eta': 0.3104945119630755, 'max_depth': 6, 'min_child_weight': 0.5760893059654519, 'subsample': 0.8624876158868648, 'colsample_bynode': 0.3992038326943794}. Best is trial 0 with value: 557.6661985761706.


...

[0]	train-rmse:5956.70283+5.44547	train-mae:5812.89385+6.42136	test-rmse:5956.69126+22.07029	test-mae:5812.89840+25.82592
[100]	train-rmse:553.12120+4.63808	train-mae:362.84164+1.89054	test-rmse:578.25642+14.62326	test-mae:375.84665+6.09475
[199]	train-rmse:506.87386+5.38888	train-mae:319.40346+2.22136	test-rmse:550.39763+16.99984	test-mae:341.87216+6.45347


[I 2023-05-03 17:54:32,949] Trial 198 finished with value: 550.3976330657202 and parameters: {'eta': 0.035097215892336824, 'max_depth': 5, 'min_child_weight': 0.0371955373094968, 'subsample': 0.931843633216609, 'colsample_bynode': 0.5294664354178588}. Best is trial 84 with value: 549.1448623889361.


[0]	train-rmse:5887.81645+5.36362	train-mae:5744.66313+6.33625	test-rmse:5887.85109+21.93702	test-mae:5744.69991+25.73423
[100]	train-rmse:537.44268+4.58548	train-mae:338.29530+1.69742	test-rmse:555.62726+16.54711	test-mae:347.76053+6.51967
[199]	train-rmse:521.11254+4.90518	train-mae:328.02723+1.77433	test-rmse:550.76826+17.27045	test-mae:343.56654+6.83528


[I 2023-05-03 17:54:35,682] Trial 199 finished with value: 550.7682626775671 and parameters: {'eta': 0.04644653515907362, 'max_depth': 4, 'min_child_weight': 0.12349045663015522, 'subsample': 0.8634730535729698, 'colsample_bynode': 0.507472788839232}. Best is trial 84 with value: 549.1448623889361.


Number of finished trials: 200
Best trial:
  Value: 549.1448623889361
  Params: 
    eta: 0.03921743172707871
    max_depth: 5
    min_child_weight: 0.034046522113062964
    subsample: 0.9194272319494144
    colsample_bynode: 0.6264947156795478


In [19]:
bestParams = study.best_trial.params
bestParams["tree_method"]= "gpu_hist"
bestParams["objective"]= "reg:squarederror"
model = xgb.train(
    params=bestParams,
    dtrain=dtrain,
    num_boost_round=boostrounds
)

blueberry_test = pd.read_csv("/home/rainer/Downloads/ML_datasets/blueberry_test.csv")
blueberry_test["dataset"]=1 #this data is also from synthetic dataset
ids = blueberry_test["id"]
blueberry_test = blueberry_test.drop("id", axis=1)
blueberry_finalPred_matrix = xgb.DMatrix(blueberry_test, enable_categorical=False)
predictions = model.predict(blueberry_finalPred_matrix)
predictionsDf = pd.DataFrame(predictions, columns=["yield"])
predictionsDf["id"]=ids


In [20]:
predictionsDf

,yield,id
0,4256.781250,15289
1,6203.761230,15290
2,7152.280273,15291
3,4730.333984,15292
4,3866.606445,15293
...,...,...
10189,5447.820801,25478
10190,5687.092773,25479
10191,6495.664551,25480
10192,4439.635254,25481


In [21]:
predictionsDf.to_csv('/home/rainer/Downloads/ML_datasets/blueberry_test_prediction.csv', columns=['id','yield'], index=False)